# Tutorial 4: Running the model on downscaled climate projections

### Outline:

* Imports, including library code from previous steps
* More in-depth exploration of the downscaled CMIP6 data
* Loading the trained model
* Setting up an inference pipeline for multiple projections
* Analysis

In [1]:
import os
import dask
import time
import torch
import torchdata
import intake
import regionmask
import xbatcher
import zen3geo as zg
import xarray as xr
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import warnings

from torch import nn
from tqdm.autonotebook import tqdm
from functools import partial
from dask.distributed import Client, LocalCluster
from torchdata.datapipes import functional_datapipe
from torchdata.datapipes.iter import IterDataPipe
from torchdata.datapipes.utils import StreamWrapper
from torchdata.dataloader2 import DataLoader2
from torch.utils.data import DataLoader
from dask.diagnostics import ProgressBar

warnings.filterwarnings('ignore')
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float32

In [2]:

cat = intake.open_esm_datastore(
  'https://cpdataeuwest.blob.core.windows.net/cp-cmip/version1/catalogs/global-downscaled-cmip6.json'
)

In [3]:
pr_data = cat.search(timescale='day', variable_id='pr')

In [5]:
pr_data

,unique
activity_id,2
institution_id,6
source_id,5
experiment_id,4
member_id,1
timescale,1
variable_id,1
method,5
downscaled_daily_data_uri,33
version,1


In [ ]:
#dask.config.set(**{'array.slicing.split_large_chunks': False})
#cluster = LocalCluster
#    n_workers=24,
#    threads_per_worker=1,
#    memory_limit='6GB',
#    dashboard_address=':2345'
#)
#client = Client(cluster)
#client

In [4]:
cat

,unique
activity_id,2
institution_id,7
source_id,6
experiment_id,4
member_id,1
timescale,3
variable_id,3
method,5
downscaled_daily_data_uri,339
version,1


In [ ]:
cat = intake.open_esm_datastore(
  'https://cpdataeuwest.blob.core.windows.net/cp-cmip/version1/catalogs/global-downscaled-cmip6.json'
)

cat_subset = cat.search(
    method="GARD-SV",
    source_id="CanESM5",
    experiment_id="ssp245",
    variable_id=['tasmin', 'tasmax', 'pr'],
    timescale='day',
)
dsets = cat_subset.to_dataset_dict()
met_ds = list(dsets.values())[0]#.chunk({'time': 1, 'lat': 48, 'lon': 48})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.timescale.method'
